# Prepare Input Data
We are going to train an MPNN to predict the B3LYP-level atomization energy.

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
from moldesign.score.mpnn.data import convert_nx_to_dict, make_tfrecord
from moldesign.utils.conversions import convert_string_to_nx
from moldesign.store.mongo import MoleculePropertyDB
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import tensorflow as tf
import pandas as pd
import json
import os

## Get the Data
The best copy is in our MongoDB isntance

In [3]:
mongo = MoleculePropertyDB.from_connection_info()

Get a mapping of smiles to "normal basis"

In [4]:
mongo.update_derived_properties()

In [5]:
data = mongo.get_training_set(['key', 'identifier.smiles', 'data.small_basis.neutral.xyz', 'data.small_basis.oxidized.xyz', 'oxidation_potential.smb-vacuum-vertical',
                              'oxidation_potential.smb-vacuum-no-zpe'], ['oxidation_potential.nob-vacuum-smb-geom'])
data = pd.DataFrame(data)
print(f'Loaded {len(data)} molecules')

Loaded 2960 molecules


Convert the SMILES to a networkx object

In [11]:
%%time
data['nx'] = data['identifier.smiles'].apply(convert_string_to_nx)

CPU times: user 837 ms, sys: 15.7 ms, total: 853 ms
Wall time: 852 ms


## Save the Data as TF Records
We'll make both a training, validation and test set

In [12]:
train_set, test_set = train_test_split(data, test_size=0.1, random_state=1, shuffle=True)
print(f'Set aside {len(test_set)} training entries')

Set aside 296 training entries


In [13]:
train_set, val_set = train_test_split(train_set, test_size=0.1, random_state=1, shuffle=True)
print(f'Split off {len(train_set)} training and {len(val_set)} validation entries')

Split off 2397 training and 267 validation entries


Get all of the types observed in the dataset

Save the data in TF format

In [14]:
os.makedirs('datasets', exist_ok=True)

In [15]:
for name, dataset in zip(['train', 'valid', 'test'], [train_set, val_set, test_set]):
    dataset = dataset.sample(frac=1.)  # Shuffle contents
    dataset.drop(columns=['nx']).to_csv(f'datasets/{name}.csv', index=False)
    with tf.io.TFRecordWriter(f'datasets/{name}_data.proto') as writer:
        for _, entry in tqdm(dataset.iterrows(), desc=name):
            record = convert_nx_to_dict(entry['nx'])
            record['output'] = entry['oxidation_potential.nob-vacuum-smb-geom']
            writer.write(make_tfrecord(record))

train: 2397it [00:00, 4973.69it/s]
valid: 267it [00:00, 4953.73it/s]
test: 296it [00:00, 5015.75it/s]
